In [80]:
#import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import yfinance as yf
import os
from fredapi import Fred
from dotenv import load_dotenv, dotenv_values
import datetime

In [81]:
#assigning env variables
load_dotenv()
FED_API_KEY = os.getenv('FED_API_KEY')
FED_RATES = ['DGS1', 'DGS2', 'DGS3', 'DGS5', 'DGS7', 'DGS10', 'DGS20', 'DGS30', 'DGS3MO', 'DGS6MO', 'DGS1MO', 'DGS2MO', 'DGS3MO', 'DGS6MO', 'T10YIE', 'FEDFUNDS']

fred = Fred(api_key=FED_API_KEY)

In [82]:
#fetch yfinance stock data
def get_stock_data(ticker, period='1y'):
    stock = yf.Ticker(ticker)
    stock_price = stock.history(period='1d')['Close'][0]
    hist = stock.history(period=period)
    hist['returns'] = hist['Close'].pct_change()
    volatility = hist['returns'].std() * (252 ** 0.5)
    return stock_price, volatility, hist

In [83]:
#fetch risk free rate from fred data
def get_fred_data(rate_id):
    fred = Fred(api_key=FED_API_KEY)
    risk_free_rate = fred.get_series(rate_id).iloc[-1] / 100
    return risk_free_rate

In [84]:
#black-scholes model
def black_scholes_calc(S0, K, r, T, sd, option_type='Call'):
    
    #Time to expiry is given in days, so convert to years
    T = T / 365

    #First, determine d1 and d2
    d1 = 1/(sd*np.sqrt(T)) * (np.log(S0/K) + (r+sd**2/2)*T)
    d2 = d1 - sd*np.sqrt(T)

    #Next, find norm cdf of d1 and d2
    nd1 = norm.cdf(d1)
    nd2 = norm.cdf(d2)

    n_d1 = norm.cdf(-d1)
    n_d2 = norm.cdf(-d2)

    #Then, find call and put value
    call = round(nd1*S0 - nd2*K*np.exp(-r*T), 2)
    put = round(K*np.exp(-r*T)*n_d2 - S0*n_d1, 2)

    if option_type=="Call":
        return call
    elif option_type=="Put":
        return put
    else:
        print("Wrong option type specified")

In [85]:
def get_time_period(expire_date):
    today = datetime.date.today()
    time_period = abs((expire_date - today).days)
    return time_period

In [89]:
#data input
ticker = 'AAPL'
strike_price = float(18)
expire_date = datetime.date.today()
risk_free_rates = FED_RATES[5]
opt_type = 'Call'

if ticker:
    apple = yf.Ticker("AAPL")
    price = apple.history(period="1d")['Close'][0]
    #stock_price, volatility, hist = get_stock_data(ticker)
else:
    print('Please enter a valid stock ticker symbol.')

if expire_date:
    time = get_time_period(expire_date)
else:
    print('Please enter a valid date.')

if risk_free_rates:
    rate = get_fred_data(risk_free_rates)


/var/folders/_8/wd_8wp492y9bssv94k3bcft00000gn/T/ipykernel_62182/343257344.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  price = apple.history(period="1d")['Close'][0]


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1002)>

In [ ]:
if ticker and stock_price and rate and time and volatility:
    value = black_scholes_calc(stock_price, strike_price, rate, time, volatility, opt_type)
    print(f"The value of your option is ${value}")
else:
    print('Error. Please try again.')